# Traditional Treatment Effect Estimation

In this notebook we introduce "Bread and Butter" techniques for causal effect estimation. At the end of the causal inference  teaching block (notebook 5), we then introduce state of the art methods. 

# 1. Generic Meta-Learners

The following are the **"traditional"** models for **causal effect estimation**, that have been used for **>40 years**. They are **generic** in the sense that all of them employ a regression method, **without specifying which method is to be used**. Thus they can be used with linear regression, logistic regression, LASSO, Random Forests and many others.


As pointed out before, causal inference is essentially a *missing data problem* and there are two ways to deal with such missing data.

When outcomes are missing for reasons beyond an investigator’s control, there are **three** different **ways** to **adjust** a **parameter estimate for covariates** that may be related both to the outcome and to missingness. One approach is to model the **relationships between the covariates and the outcome and use those relationships to predict the missing values**. Another is to model the **probabilities of missingness** given the covariates and incorporate them into a weighted or stratified estimate. A last approach is to __identify the closest factual values of interest__.


## 1.1 K-Nearest Neighbours

Idea: take the closest factual observation ob treatment and non-treatment. Closeness is determined by k-nearest neighbours.

This approach allows to derive an estimator of the CATE on an individual unit level. The Causal KNN regression method is represented in the following equation:
$\widehat{\tau}_{K}(x)=\frac{1}{K} \sum_{i \in N_{K}(x, 1)} \cdot Y_{i}-\frac{1}{K} \sum_{i \in N_{K}(x, 0)} \cdot Y_{i}$
where $Y_{i}$ represents the outcome values of the target variable and $N_{K}(x, 0)$ and $N_{K}(x, 1)$ denote the nearest neighbour units with treatment status $W_{i}=1$ and $W_{i}=0$ respectively. Since the Causal KNN algorithm takes the number of nearest neighbours $K$ as a freely selectable parameter, it is necessary to find a way to choose the optimal $K$ value. 


K-nearest neighbours usually takes Eucledian Distance as a metric.


## 1.2 Propensity score weighing/ propensity score matching

Introduced by;
[Rosenbaum and Donald B Rubin (1983)](https://academic.oup.com/biomet/article-pdf/70/1/41/662954/70-1-41.pdf). 

A good introduction can also be found in
[Guo et al. (2019)](https://arxiv.org/pdf/1809.09337.pdf). 

**Idea**: **estimate** the effect of a treatment, policy, or other **intervention** by **accounting for the covariates that predict receiving the treatment**.

**Propensity Score**: A propensity score is the **probability of a unit** (e.g., person, classroom, school) being **assigned to a particular treatment** given a set of observed covariates: 

$$p(x) = P(T = 1 | X = x) = E[T | X = x]$$

Using this propensity score estimate, the idea is to create a pseudo-population by
weighting each sample with the inverse propensity score (IPS-Weighting, or IPSW)


According to Rosenbaum and Rubin (1983), the treatment **assignment** is **independent
of the observed covariates** given the propensity score, $X \perp T | p(x)$, if the **unconfoundedness assumption holds**. This theorem is also known as the **Sufficiency of the Propensity Score**. Essentially it says that **if** the **covariates are sufficient for adjustment then so is the propensity score**.

We can **use any generic supervised machine learning method** or regression algorithm
to **estimate p(x)**. Assuming that our propensity score estimate $\hat{p}(x)$ equals the true
propensity score p(x), we get

$$\mathbb{E} \bigg{[} \frac{TY}{p(x)}\bigg{]} =\mathbb{E} \bigg{[} \frac{TY(1)}{p(x)}\bigg{]} $$ - using assumption of stable unit treatment value 

$$= \mathbb{E} \bigg{[} \mathbb{E} \bigg{[} \frac{TY(1)}{p(x)}| Y(1), X\bigg{]}\bigg{]}$$ 

$$= \mathbb{E} \bigg{[}  \frac{Y(1)}{p(x)}\mathbb{E} \bigg{[} T|Y(1), X\big{]}\bigg{]}$$ 

$$= \mathbb{E} \bigg{[} \frac{Y(1)}{p(x)} \mathbb{E} \big{[} T|  X\big{]}\bigg{]}$$ - using the unconfoundeness assumption

$$= \mathbb{E} \bigg{[} \frac{Y(1)}{p(x)} p(x) \bigg{]}= \mathbb{E}\big{[}Y(1)\big{]}$$ using $p(x)=\mathbb{E}\big{[}Y(1)\big{]}$


We can follow the same steps to show that 


$ \mathbb{E}[  \frac{(1-T)Y}{(1-p(x)} ]   = \mathbb{E}[Y (0)]$.

Then we can again simply estimate the average causal effect 

$ τ = \mathbb{E}[Y (1)]−E[Y (0)]$. 
 
 
 For a finite sample
$S$, with $|S| = n$, this can be written, using propensity score estimate $\hat{p}$, as

$$ \hat{τ}_{IPW} = n^{−1} \sum_{i=1}^{n} \frac{T_i Y_i}{\hat{p}(x)}- n^{−1} \sum_{i=1}^{n} \frac{(1-T_i)Y_i}{1-\hat{p}(x)}$$


This gives a **higher weight** to **instances** that are **underrepresented**. This
means that in the first summand, treated instances whose predicted probability of
treatment is very low, are weighted very high. The second summand achieves the
same for control instances, in reverse logic. Notice that we are dependent on the
quality of our propensity estimate for this method to work. The propensity score
model has to approximate the true assignment mechanism for the weighting to be
meaningful. Thus, only if the chosen regression model equals the true regression,
our propensity score weighted estimate recovers the true average treatment effect.


**An intuitive illustration of propensity score matching using nearest neighbours**


<img src="https://www.researchgate.net/profile/Donald_Rubin3/publication/228519896/figure/fig3/AS:301801178976267@1448966416950/11-Nearest-neighbor-matching-on-propensity-score-followed-by-subclassification-Black_W640.jpg" width=75%>


1:1 Nearest neighbor matching on propensity score followed by subclassification. Black units were matched; grey units were unmatched. Subclasses indicated by vertical lines

## 1.3 Conditional Mean Regression / regression adjustment

### 1.3.1 S-Learners (Single-Learners)

Use single supervised machine learning technique to estimate combined response function

$$µ(x; t) := \mathbb{E}[Y | X = x; T = t]:$$

Let $\hat{µ}$ be the estimator of µ. We can then estimate the conditional treatment effect as

$$τ(x) = \hat{µ} (x; 1) − \hat{µ}(x; 0)$$



### 1.3.2 T-Learners (Treatment-Learners)
T-Learners on the other hand use **two estimators** to perform the same task. Essentially we split the dataset into treated and control and learn an outcome regression on each of the subsets. Thus, we learn

$$µ_0(x) = \mathbb{E}[Y | X = x; T = 1],$$

using observations in the treated group. Similarly we learn

$$µ_1(x) = \mathbb{E}[Y | X = x, T = 0],$$

using observations in the control group. Finally, we use the estimates $\hat{µ}_1$ and $\hat{µ}_0$ to get

$$2\hat{τ} (x) = \hat{µ}_1(x) − \hat{µ}_0(x):$$

A key advantage of T-learners is that we can use **different methods** to **estimate** $µ_1$ and
$µ_0$ and thus adapt to the structure of the data if necessary. Also, a notable side effect
of using a S-Learner with Linear Regression is that it cannot capture heterogeneous
effects because the coefficient $α_T$remains fixed for any predicted instance.

**Why does it work?**
Consider the simple example where we use a linear regressor with the S-learner method and estimate

$$\mathbb{E}[Y | X, T ] ≈ α_0 + α_T + X^T α_X$$.

Then, intuitively 

$$\hat{µ}(x; 1) − \hat{µ}(x; 0) $$

is equal to the coefficient $α_T$ of the treatment
indicator. This is to say that $α_T$ is the average treatment effect. More formally and
generalized, the efficacy of regression adjustment is based on the following derivation:

$$\mathbb{E} [\mathbb{E}[Y | X; T = 1]] = \mathbb{E} [\mathbb{E}[Y (1) | X; T = 1]]$$ using the SUTVA assumption

$$= \mathbb{E}  [\mathbb{E} [Y (1) | X]] = \mathbb{E} [Y (1)]$$ using the unconfoundedness assumption


However, that is only **true if the linear regression is actually the true regression of
the outcome** on covariates and treatment. Also, as always, unconfoundedness and
consistency constraints from Assumptions must hold.

It's important to note that, just like propensity score weighting, outcome regression
is dependent on the regression model we choose.

## 1.4 Doubly Robust Estimators

The core idea behind Doubly Robust
Estimators (DRE) is to combine IPSW and T-Learners in a smart way, such that
only one of two regression models has to be chosen correctly. Formally, the double
robust estimator for finite samples is defined as

$$ \hat{τ}_{DR} =n^{−1} \sum_{i=1}^{n}  \bigg{[}  \frac{T_i Y_i}{p(X_i)} -  \frac{T_i -p(X_i)}{p(X_i)} µ_1(X_i)  \bigg{]} - n^{-1}      
\sum_{i=1}^{n}  \bigg{[}  \frac{(1-T_i)Y_i}{1-p(X_i)} -  \frac{T_i-p(X_i)}{p(X_i)} µ_1(X_i)  \bigg{]}
$$


https://www4.stat.ncsu.edu/~davidian/double.pdf

Following .[Davidian (2007)](https://www4.stat.ncsu.edu/~davidian/double.pdf). , one can split this term into an estimation for $\mathbb{E}[Y (1)]$ and $\mathbb{E}[Y (0)]$. The two separate forms can then be written as the quantity they estimate plus a residual which must be zero for the estimator to work perfectly. We can derive

$$ µ_{DR,1} ≈ \mathbb{E}[Y (1)] + \mathbb{E} \bigg{[} \frac{T-p(x)}{p(X)}· (Y (1) − µ_1(X)) \bigg{]}$$

where $\mathbb{E}[Y (1)]$ is the target quantity. For the second term to be zero, either the
propensity score or the outcome regression has to match the true regression. As a
user of a doubly robust estimator, we have to choose one model for the propensity
and one for the outcomes. Only one of the two, however, must be specified correctly
for the estimation to work. 


[See here for more detais: Demystifying Double Robustness: A Comparison of Alternative Strategies for Estimating a Population Mean from Incomplete Data](https://projecteuclid.org/euclid.ss/1207580167) 




# 2. Causal Forrest

The original paper was by Wagner and Athey (2017) **"Estimation and Inference of Heterogeneous Treatment Effects using Random Forests"**, [here](https://arxiv.org/abs/1510.04342). 

## 2.1 Basic Idea

-	Create **leaves** that are **homogenous** in terms of the **effect** of the **independent variables** (except for the treatment) on the **outcome**. 
-	Then, this is like being **statistically "randomized"** **within** a **leave**.
-	If the **leaves** are of **"sufficient size"** and contain both **treatment** and **non-treatment**, we can **estimate** the **difference** between **treatment and non-treatment** **within** this **leave**  
-	Use **honest trees** (hold out sample to predict causal effect) to do so - data for splitting into leaves and estimating causal effects 

## 2.2 Important Idea of Random Forests vs. Traditional Clustering

-	**Random forests** are **related to kernels and nearest-neighbour** methods in that they make **predictions** using a **weighted average of  "nearby" observations**;
-	However, random forests differ in that they have a **data-driven way to determine which nearby observations receive more weight**, something that is especially important in environments with many covariates or complex interactions among covariates. (Important dimensions can be sliced more finely) 
-	Particularly important for **hight dimensional spaces**, where we need much more data otherwise


## 2.3 Treatment estimation with unconfoundedness

- At a high level, trees and forests can be thought of as nearest neighbor methods with an adaptive neighborhood metric. 
- Given a **test point x**, classical methods such as k-nearest neighbors seek the k closest points to x **according to some pre-specified distance measure**, e.g., Euclidean distance. 
- In contrast, tree-based methods also seek to find training examples that are close to x, but **now closeness is defined with respect to a decision tree**, and the 5 closest points to x are those that **fall in the same leaf as it**. 
- The advantage of trees is that their leaves can be **narrower along the directions where the signal is changing fast** and wider along the other directions, potentially leading a to a substantial **increase in power** when the **dimension of the feature space** is even **moderately large**.
- Suppose first that we only observe independent samples $(X_i; Y_i)$, and want to build a CART regression tree.
- We start by **recursively splitting** the **feature space** until we have **partitioned** it **into a set of leaves L**, each of which only contains a few training samples. Then, given a test point x, we **evaluate the prediction $\hat{µ}(x)$ by identifying the leaf L(x) containing x and setting**:


$$(4) \hat{µ}(x)=\frac{1}{|\{i:X_i \epsilon L(X) \}|} \sum_{\{i:X_i \epsilon L(X) \}} Y_i $$

 
 
-  This strategy is well-motivated if we believe the leaf L(X) to be small enough that the response $Y_i$ inside the leaf are roughly identically distributed. 
- In the context of causal trees, we analogously want to think of the leaves as small enough that $(Y_i,W_i)$ pairs corresponding to the indices $i$ for which $i \epsilon L(x)$ act as **though they had come from a randomized experiment**. 
- Then, it is natural to estimate the treatment effect for any $ x \epsilon L$ as

$$(5) \hat{τ}(x)=\frac{1}{|\{i:W_i =1, X_i \epsilon L \}|} \sum_{\{i:W_i =1, X_i \epsilon L \}} Y_i - \frac{1}{|\{i:W_i =0, X_i \epsilon L \}|} \sum_{\{i:W_i =0, X_i \epsilon L \}} Y_i $$



 
<img src="http://drive.google.com/uc?export=view&id=1DYf65tDA0vl-cm_fty7ooLurQ3hDiY-w" width=75%>
 

**Some additional useful links to background material for causal forests**

[Towards Data Science - Estimation and Inference of Heterogeneous Treatment Effects using Random Forests](https://towardsdatascience.com/estimation-and-inference-of-heterogeneous-treatment-effects-using-random-forests-paper-review-c26fb97c96b7) 

[EXPLICITLY OPTIMIZING ON CAUSAL EFFECTS VIA THE CAUSAL RANDOM FOREST: A PRACTICAL INTRODUCTION AND TUTORIAL](https://www.markhw.com/blog/causalforestintro) 

[Causal forests: A tutorial in high-dimensional causal inference](https://scholar.princeton.edu/sites/default/files/bstewart/files/lundberg_methods_tutorial_reading_group_version.pdf) 

